In [1]:
import numpy as np
import pandas as pd
from utils import dictionaries, Eurostat

In [2]:
filename="../data/Eurostat Enterprise Stats Latest (inc.2023) (1).xlsx"

In [3]:
sheets=pd.ExcelFile(path_or_buffer=filename).sheet_names

In [4]:
eurostat=Eurostat()

In [5]:
sheet_flags=[]
for sheet in sheets:
    sheet_flags.append(eurostat.sheet_finder(sheet=sheet,filepath=filename))

In [6]:
dfs=[]
for sheet,annotation in zip(sheets,sheet_flags):
    if annotation:
        dfs.append(eurostat.sheet_reader(sheet=sheet,filepath=filename))

In [7]:
eurostat_data=pd.concat(dfs,axis=0)

In [8]:
eurostat_data["s_level_segment"]=eurostat_data["segment"].apply(
    lambda segment: dictionaries["eurostat_segments"].get(segment)
)

In [9]:
eurostat_data=(
    eurostat_data
    .groupby(by=["country","s_level_segment"])
    [["2021","2022","2023"]]
    .sum()
    .reset_index()
)

In [10]:
segment_pairs=[
    ("s5-s6","s5"),
    ("s5-s6","s6"),
    ("s7+","s7"),
    ("s7+","s8"),
    ("s7+","s9"),
    ("s7+","s10")
]

In [11]:
non_eu=pd.read_excel(
    io="../data/Data Landscaping Master (Draft) - PQ WIP.xlsx",
    sheet_name="Non-Eurostat Hardcoded",
    skiprows=6
)

In [12]:
non_eu["s_level_segment"]=non_eu["Size class"].apply(
    lambda segment: dictionaries["eurostat_segments"].get(segment)
)
non_eu=non_eu.drop(columns=["Size class"])

In [13]:
non_eu=(
    non_eu
    .groupby(by=["country","s_level_segment"])
    [["2021","2022","2023"]]
    .sum()
    .reset_index()
)

In [14]:
mapped_non_eu=eurostat.segment_mapper(
        df_to_map=non_eu,
        segment_pairs=segment_pairs
    )
mapped_eu=eurostat.segment_mapper(
    df_to_map=eurostat_data,
    segment_pairs=segment_pairs
)

In [15]:
eu_stat_processed=pd.concat([
    mapped_eu[mapped_eu["country"]!="Albania"],
    mapped_non_eu
])

In [17]:
eu_stat_processed.to_csv("../data/output_2.csv",index=False)